# ReadMe.md

# Import

In [2]:
import numpy as np
import json
import os, os.path
import keras
import tensorflow as tf
import warnings
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import cv2
warnings.filterwarnings('ignore')
from keras.layers import Conv2D, Activation, MaxPool2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# YOLO library ( Image Detect )
---
1. opencv로 image read
2. useyolo함수에 img넣고 return
3. 출력

In [3]:
Dir = "D:/hackathon/"
net = cv2.dnn.readNet(Dir+"yolov3.weights", Dir+"yolov3.cfg")
classes = []
with open(Dir+"coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

def useyolo(img) :
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # 좌표
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
    return img

# ProtoType ( 전체 카테고리 )
---


In [16]:
img_dir = 'D:/hackathon/이미지사운드매칭_해커톤_데이터셋/이미지/원천데이터/'
for b_cate in os.listdir(img_dir) :
    print(f"===큰 주제 : {b_cate}===")
    print("-소주제")
    s_label = list()
    f_cnt = 0
    for s_cate in os.listdir(img_dir+b_cate+'/') :
        s_label.append(s_cate)
        f_cnt += len(os.listdir(img_dir+b_cate+'/'+s_cate+'/'))
    print(s_label)
    print("-image")
    datagen=ImageDataGenerator(rescale=1./255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest',
                               validation_split=0.2)
    train_dataset = datagen.flow_from_directory(directory=img_dir+b_cate+'/',
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="training",
                                                 class_mode='categorical',
                                               batch_size=20)
    validation_dataset = datagen.flow_from_directory(directory=img_dir+b_cate+'/',
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="validation",
                                                 class_mode='categorical',
                                                    batch_size=20)
    
#     print(f_cnt-(f_cnt//5), f_cnt//5)
#     continue
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=(150,150,3)))
    model.add(keras.layers.MaxPooling2D(2,2))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Conv2D(64, kernel_size=(3,3),activation='relu'))
    model.add(keras.layers.MaxPooling2D(2,2))
    model.add(keras.layers.Conv2D(64, kernel_size=(3,3),activation='relu'))
    model.add(keras.layers.MaxPooling2D(2,2))
    model.add(keras.layers.Dropout(0.2))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512,activation='relu'))
    model.add(keras.layers.Dense(len(s_label),activation='softmax'))
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
    with tf.device("/device:GPU:0") :
        model.fit_generator(train_dataset,
                           steps_per_epoch=(f_cnt-(f_cnt//5))//20,
                           validation_data=validation_dataset,
                           epochs=20,
                           validation_steps=(f_cnt//5)//20)
    model.save_weights(b_cate+'.h5')

===큰 주제 : 교통===
-소주제
['경고음', '선박 운항', '차량 공회전', '차량 주행']
-image
Found 812 images belonging to 4 classes.
Found 203 images belonging to 4 classes.
Epoch 1/20
40/40 [==============================] - 226s 6s/step - loss: 1.7155 - accuracy: 0.3611 - val_loss: 1.2393 - val_accuracy: 0.3850
Epoch 2/20
40/40 [==============================] - 167s 4s/step - loss: 0.8592 - accuracy: 0.5846 - val_loss: 0.9918 - val_accuracy: 0.5400
Epoch 3/20
40/40 [==============================] - 169s 4s/step - loss: 0.7376 - accuracy: 0.6288 - val_loss: 1.0144 - val_accuracy: 0.5300
Epoch 4/20
40/40 [==============================] - 166s 4s/step - loss: 0.6641 - accuracy: 0.6427 - val_loss: 1.4377 - val_accuracy: 0.4300
Epoch 5/20
40/40 [==============================] - 160s 4s/step - loss: 0.6173 - accuracy: 0.7172 - val_loss: 1.2190 - val_accuracy: 0.5050
Epoch 6/20
40/40 [==============================] - 162s 4s/step - loss: 0.6021 - accuracy: 0.7096 - val_loss: 1.1618 - val_accuracy: 0.5350
Epoch 7/